<p style="align: center;"><img src="https://static.tildacdn.com/tild6636-3531-4239-b465-376364646465/Deep_Learning_School.png" width="400"></p>

# Домашнее задание. Обучение языковой модели с помощью LSTM (10 баллов)

Э
В этом задании Вам предстоит обучить языковую модель с помощью рекуррентной нейронной сети. В отличие от семинарского занятия, Вам необходимо будет работать с отдельными словами, а не буквами.


Установим модуль ```datasets```, чтобы нам проще было работать с данными.

Импорт необходимых библиотек

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from datasets import load_dataset
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
import nltk

from collections import Counter
from typing import List

import seaborn
seaborn.set(palette='summer')

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/dmitry/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Подготовка данных

Воспользуемся датасетом imdb. В нем хранятся отзывы о фильмах с сайта imdb. Загрузим данные с помощью функции ```load_dataset```

In [4]:
# Загрузим датасет
dataset = load_dataset('imdb')

### Препроцессинг данных и создание словаря (1 балл)

Далее вам необходмо самостоятельно произвести препроцессинг данных и получить словарь или же просто ```set``` строк. Что необходимо сделать:

1. Разделить отдельные тренировочные примеры на отдельные предложения с помощью функции ```sent_tokenize``` из бибилиотеки ```nltk```. Каждое отдельное предложение будет одним тренировочным примером.
2. Оставить только те предложения, в которых меньше ```word_threshold``` слов.
3. Посчитать частоту вхождения каждого слова в оставшихся предложениях. Для деления предлоения на отдельные слова удобно использовать функцию ```word_tokenize```.
4. Создать объект ```vocab``` класса ```set```, положить в него служебные токены '\<unk\>', '\<bos\>', '\<eos\>', '\<pad\>' и vocab_size самых частовстречающихся слов.   

In [5]:
sentences = []
word_threshold = 32

for sentence in tqdm(dataset['unsupervised']['text']):
    sentences.extend(
        [x.lower() for x in sent_tokenize(sentence) if len(word_tokenize(x)) < word_threshold]
    )

  0%|          | 0/50000 [00:00<?, ?it/s]

In [6]:
print("Всего предложений:", len(sentences))

Всего предложений: 392618


In [7]:
def is_bad_token(word):
    return False
    #return (len(word) == 2 and word[0] == word[1]) or len(word) == 1

Посчитаем для каждого слова его встречаемость.

In [8]:
words = Counter()

for sentence in tqdm(sentences):
    for word in word_tokenize(sentence):
        if not is_bad_token(word):
            words[word] += 1

  0%|          | 0/392618 [00:00<?, ?it/s]

Добавим в словарь ```vocab_size``` самых встречающихся слов.

In [9]:
special_tokens = ['<unk>', '<bos>', '<eos>', '<pad>']
vocab = set()
vocab_size = 40000


vocab = special_tokens + [word for word, _ in words.most_common(vocab_size)]

In [10]:
assert '<unk>' in vocab
assert '<bos>' in vocab
assert '<eos>' in vocab
assert '<pad>' in vocab
assert len(vocab) == vocab_size + 4

In [11]:
print("Всего слов в словаре:", len(vocab))

Всего слов в словаре: 40004


### Подготовка датасета (1 балл)

Далее, как и в семинарском занятии, подготовим датасеты и даталоадеры.

В классе ```WordDataset``` вам необходимо реализовать метод ```__getitem__```, который будет возвращать сэмпл данных по входному idx, то есть список целых чисел (индексов слов).

Внутри этого метода необходимо добавить служебные токены начала и конца последовательности, а также токенизировать соответствующее предложение с помощью ```word_tokenize``` и сопоставить ему индексы из ```word2ind```.

In [12]:
word2ind = {char: i for i, char in enumerate(vocab)}
ind2word = {i: char for char, i in word2ind.items()}

In [13]:
class WordDataset:
    def __init__(self, sentences):
        self.data = sentences
        self.unk_id = word2ind['<unk>']
        self.bos_id = word2ind['<bos>']
        self.eos_id = word2ind['<eos>']
        self.pad_id = word2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        tokenized_sentence = [self.bos_id]
        for word in self.data[idx]:
            if not is_bad_token(word):
                tokenized_sentence.append(word2ind.get(word, self.unk_id))
        
        tokenized_sentence.append(self.eos_id)

        return tokenized_sentence

    def __len__(self) -> int:
        return len(self.data)

In [14]:
def collate_fn_with_padding(
    input_batch: List[List[int]], pad_id=word2ind['<pad>']) -> torch.Tensor:
    seq_lens = [len(x) for x in input_batch]
    max_seq_len = max(seq_lens)

    new_batch = []
    for sequence in input_batch:
        for _ in range(max_seq_len - len(sequence)):
            sequence.append(pad_id)
        new_batch.append(sequence)

    sequences = torch.LongTensor(new_batch).to(device)

    new_batch = {
        'input_ids': sequences[:,:-1],
        'target_ids': sequences[:,1:]
    }

    return new_batch

In [15]:
train_sentences, eval_sentences = train_test_split(sentences, test_size=0.2)
eval_sentences, test_sentences = train_test_split(sentences, test_size=0.5)

train_dataset = WordDataset(train_sentences)
eval_dataset = WordDataset(eval_sentences)
test_dataset = WordDataset(test_sentences)

batch_size = 32

train_dataloader = DataLoader(
    train_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

eval_dataloader = DataLoader(
    eval_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

test_dataloader = DataLoader(
    test_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

## Обучение и архитектура модели

Вам необходимо на практике проверить, что влияет на качество языковых моделей. В этом задании нужно провести серию экспериментов с различными вариантами языковых моделей и сравнить различия в конечной перплексии на тестовом множестве.

Возмоэные идеи для экспериментов:

* Различные RNN-блоки, например, LSTM или GRU. Также можно добавить сразу несколько RNN блоков друг над другом с помощью аргумента num_layers. Вам поможет официальная документация [здесь](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)
* Различные размеры скрытого состояния. Различное количество линейных слоев после RNN-блока. Различные функции активации.
* Добавление нормализаций в виде Dropout, BatchNorm или LayerNorm
* Различные аргументы для оптимизации, например, подбор оптимального learning rate или тип алгоритма оптимизации SGD, Adam, RMSProp и другие
* Любые другие идеи и подходы

После проведения экспериментов необходимо составить таблицу результатов, в которой описан каждый эксперимент и посчитана перплексия на тестовом множестве.

Учтите, что эксперименты, которые различаются, например, только размером скрытого состояния или количеством линейных слоев считаются, как один эксперимент.

Успехов!

### Функция evaluate (1 балл)

Заполните функцию ```evaluate```

In [16]:
def evaluate(model, criterion, dataloader) -> float:
    model.eval()
    perplexity = []
    with torch.no_grad():
        for batch in dataloader:
            logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
            loss = criterion(logits, batch['target_ids'].flatten())
            perplexity.append(torch.exp(loss).item())

    perplexity = sum(perplexity) / len(perplexity)

    return perplexity

### Train loop (1 балл)

Напишите функцию для обучения модели.

In [17]:
def train_model(model, num_epochs, optimizer, criterion, train_dataloader, eval_dataloader, writer):
    for epoch in tqdm(range(num_epochs)):
        train_perplexity = []
        val_perplexity = []

        train_losses = []
        val_losses = []

        train_predictions = []
        train_target = []

        model.train()
        for batch in tqdm(train_dataloader, desc=f'Training epoch {epoch}'):
            optimizer.zero_grad()
            logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
            loss = criterion(logits, batch['target_ids'].flatten())
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())
            train_perplexity.append(torch.exp(loss).item())
            train_predictions.append(logits.argmax(dim=1))
            train_target.append(batch['target_ids'].flatten())

        train_predictions = torch.cat(train_predictions)
        train_target = torch.cat(train_target)
    
        val_predictions = []
        val_target = []

        model.eval()
        with torch.no_grad():
            for batch in tqdm(eval_dataloader, desc=f'Evaluating epoch {epoch}'):
                logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
                val_predictions.append(logits.argmax(dim=1))
                val_target.append(batch['target_ids'].flatten())
                loss = criterion(logits, batch['target_ids'].flatten())
                val_losses.append(loss.item())
                val_perplexity.append(torch.exp(loss).item())

        val_predictions = torch.cat(val_predictions)
        val_target = torch.cat(val_target)

        writer.add_scalar('loss/train', np.mean(train_losses), epoch)
        writer.add_scalar('loss/val', np.mean(val_losses), epoch)
        writer.add_scalar('accuracy/train', (train_predictions == train_target).float().mean().item(), epoch)
        writer.add_scalar('accuracy/val', (val_predictions == val_target).float().mean().item(), epoch)
        writer.add_scalar('perplexity/train', sum(train_perplexity) / len(train_perplexity), epoch)
        writer.add_scalar('perplexity/val', sum(val_perplexity) / len(val_perplexity), epoch)

### Первый эксперимент (2 балла)

Определите архитектуру модели и обучите её.

In [18]:
class LanguageModel(nn.Module):
    def __init__(self, hidden_dim: int, vocab_size: int):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)
        
        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(0.1)

    def forward(self, input_batch: torch.Tensor) -> torch.Tensor:
        embeddings = self.embedding(input_batch)
        output, _ = self.rnn(embeddings)
        output = self.non_lin(output)
        output = self.linear(output)
        output = self.dropout(output)
        output = self.non_lin(output)
        output = self.projection(output)

        return output

In [19]:
from torch.utils.tensorboard import SummaryWriter

model = LanguageModel(hidden_dim=256, vocab_size=len(vocab)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
writer = SummaryWriter('.logs/LanguageModel')

2025-07-25 19:56:31.435572: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753462592.446822   23363 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753462592.731070   23363 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753462594.964356   23363 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753462594.964411   23363 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753462594.964416   23363 computation_placer.cc:177] computation placer alr

In [20]:
num_epochs = 15

train_model(model, num_epochs, optimizer, criterion, train_dataloader, eval_dataloader, writer)

  0%|          | 0/15 [00:00<?, ?it/s]

Training epoch 0:   0%|          | 0/9816 [00:00<?, ?it/s]

Evaluating epoch 0:   0%|          | 0/6135 [00:00<?, ?it/s]

Training epoch 1:   0%|          | 0/9816 [00:00<?, ?it/s]

Evaluating epoch 1:   0%|          | 0/6135 [00:00<?, ?it/s]

...

Evaluating epoch 12:   0%|          | 0/6135 [00:00<?, ?it/s]

Training epoch 13:   0%|          | 0/9816 [00:00<?, ?it/s]

Evaluating epoch 13:   0%|          | 0/6135 [00:00<?, ?it/s]

Training epoch 14:   0%|          | 0/9816 [00:00<?, ?it/s]

Evaluating epoch 14:   0%|          | 0/6135 [00:00<?, ?it/s]

In [26]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [27]:
%tensorboard --logdir .logs

Reusing TensorBoard on port 6006 (pid 26897), started 0:01:40 ago. (Use '!kill 26897' to kill it.)

In [23]:
def generate_sequence(model, starting_seq: str, max_seq_len: int = 32) -> str:
    device = 'cpu'
    model = model.to(device)
    input_ids = [word2ind['<bos>']] + [
        word2ind.get(word, word2ind['<unk>']) for word in word_tokenize(starting_seq) if not is_bad_token(word)]
    input_ids = torch.LongTensor(input_ids).to(device)

    model.eval()
    with torch.no_grad():
        for i in range(max_seq_len):
            next_word_distribution = model(input_ids)[-1]
            next_word = next_word_distribution.squeeze().argmax()
            input_ids = torch.cat([input_ids, next_word.unsqueeze(0)])

            if next_word.item() == word2ind['<eos>']:
                break

    words = [ind2word[idx.item()] for idx in input_ids]

    return ' '.join(words)

In [24]:
generate_sequence(model, starting_seq='hello')

'<bos> hello <eos>'

### Второй эксперимент (2 балла)

Попробуйте что-то поменять в модели или в пайплайне обучения, идеи для экспериментов можно подсмотреть выше.

In [25]:
# Проведите второй эксперимент

### Отчет (2 балла)

Опишите проведенные эксперименты. Сравните перплексии полученных моделей. Предложите идеи по улучшению качества моделей.